<a href="https://www.kaggle.com/code/adends/neural-network-with-embeddings-for-house-prices?scriptVersionId=169635138" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# House Prices - Advanced Regression Techniques
## Introduction
- This kaggle notebook will be for the kaggle competition **House Prices - Advanced Regression**. We will be exploring the dataset with visualizations and analysis. We will then preprocess it for our model. We will then create a PyTorch Neural Network with embedding layers for the categorical features.

## Table of Contents
- [Exploratory Data Analysis (EDA)](#Exploratory-Data-Analysis)
- [Data Preprocessing](#Data-Preprocessing)
  - [Feature Engineering](#Feature-Engineering)
  - [Encoding](#Encoding)
  - [Feature Selection](#Feature-Selection)
  - [Imputation](#Imputation)
  - [Testing Dataset Preprocessing](#Testing-Dataset-Preprocessing)
- [Model Building](#Model-Building)
    - [Creating Tensors](#Creating-Tensors)
    - [Embedding Sizes](#Embedding-Sizes)
    - [Model Architecture](#Model-Architecture)
    - [Model Evaluation and Submission](#Model-Evaluation-and-Submission)




# Exploratory Data Analysis

In [ ]:
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
df.head()

From this head snippet, we can see that we have some columns (Alley, PoolQC, MiscFeature) likely have a high number of missing values. We'll be sure to investigate those.

In [ ]:
df.info()

We can see that we have quite a few columns (1600, **81**). A lot of these are probably irrelevant so we need to filter those out for later results. Attempts with all of these columns usually results in a model with an NaN loss, meaning no predictions can really be made. We also see we have quite a few categorical features. Our stratedgy for these will be embedding layers.

In [ ]:
df.describe()

In [ ]:
# Get all columns that contain missing values and plot with barplot
non_full_cols = df.isna().sum()
non_full_cols = non_full_cols[non_full_cols != 0]

fig, ax = plt.subplots(figsize=(10,5))
non_full_cols.plot(kind='bar', ax=ax, color='coral')

ax.set_title('Number of Missing Values per Column')
ax.set_xlabel('Columns')
ax.set_ylabel('Number of Missing Values')

plt.xticks(rotation=45, ha='right') 
plt.show()

Now we get a better look into the completeness of our dataset. We have a couple of extreme columns with nearly all missing values (Remember original dataset was 1600 entries). For these extremely incomplete rows we are going to remove them. For the gray area columns like `LotsFrontage` and `FirePlaceQu`, we will keep them for now.

In [ ]:
# Identifying mostly complete columns (low_missing_cols) and splitting them into Categorical and Numerical
low_missing_cols = ['MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'GarageType','GarageFinish', 'GarageYrBlt', 'GarageQual', 'GarageCond']
filtered_df = df[low_missing_cols]

numerical_missing_cols = filtered_df.select_dtypes(include='number').columns.tolist()
categorical_missing_cols = filtered_df.select_dtypes(include='object').columns.tolist()

In [ ]:
# Ensure the selected categorical columns are indeed categorical
df[categorical_missing_cols]

In [ ]:
# Identify mostly incomplete columns (Missing values greater than half of total)
missing_counts = df.isna().sum()
cols_with_high_missing = missing_counts[missing_counts > len(df) // 2].index
high_missing_cols = df[cols_with_high_missing] 

high_missing_cols

# Data Preprocessing

In [ ]:
# Drop the high missing cols from before aswell as Id (Identifiers not relevant)
df.drop(high_missing_cols, axis=1, inplace=True)
df_test.drop(high_missing_cols, axis=1, inplace=True)
df.drop(['Id'], axis=1, inplace=True)

## Feature Engineering
With more domain knowledge you could engineer more features based on the existing ones. For this notebook we will keep it simple and are only transforming certain features.
- Replace `YearBuilt` and `GarageYrBlt` with `Age` and `GarageYrBlt`
- New age features are time differences between Today and Year Built for more domain relevance


In [ ]:
df['Age']=datetime.datetime.now().year-df['YearBuilt']
df['GarageAge'] = datetime.datetime.now().year-df['GarageYrBlt']
df.drop(['YearBuilt','GarageYrBlt'], axis=1, inplace=True)

df_test['Age']=datetime.datetime.now().year-df_test['YearBuilt']
df_test['GarageAge'] = datetime.datetime.now().year-df_test['GarageYrBlt']
df_test.drop(['YearBuilt','GarageYrBlt'], axis=1, inplace=True)

In [ ]:
# Plotting distribution for Target Variable (SalePrice)
fig, ax = plt.subplots(figsize=(10,5))

df['SalePrice'].plot(kind='kde',ax=ax, color='coral',)
df['SalePrice'].hist(density=True, ax=ax, color='teal', alpha=0.5, bins=40)  

ax.set_xlabel('Sale Price (Target)')
ax.set_ylabel('Density')
ax.set_title('Sale Price Density')

plt.show()

From this plot, we can observe that our target variable, Sale Price, is normally distributed. This indicates that most houses are priced around a central value, suggesting a similar range of sale prices across the dataset. The normal distribution also implies that there is not a significant amount of extreme variation or outliers in sale prices.

In [ ]:
numeric_df = df.select_dtypes(include='number')
numeric_features = numeric_df.columns
numeric_df.hist(figsize=(16, 22), bins=40, color='coral')

From these plots we can see that of the numerical features we have the continous variables indicated by the more complete graphs (SalePrice, Age, GarageAge, GarageArea, etc) and discrete indicated by the sparse graphs (GarageCars, FirePlaces, KitchenAbvGr, etc).

In [ ]:
cat_df = df.select_dtypes(include='object')
unique_counts = cat_df.nunique()

plt.figure(figsize=(10, 6))
unique_counts.plot(kind='bar', color='teal')

plt.title('Number of Unique Categories in Each Categorical Column')
plt.xlabel('Categorical Columns')
plt.ylabel('Number of Unique Categories')
plt.xticks(rotation=45, ha='right') 

plt.show()

This plot gives us the cardinality of our categorical features, most are in the 0-5 range however we have some higher ones like Neighborhood.

In [ ]:
# cat_df = pd.DataFrame(cat_df['MSZoning'])

## Encoding
Since we are going to use an embedding layer we will be using LabelEncoder. Different encoders can be experiemented with however, this one works easiest.



In [ ]:
cat_features = df.select_dtypes(include=['object']).columns  
encoders = {col: LabelEncoder().fit(df[col]) for col in cat_features}

for col in cat_features:
    # Fit the encoder including an extra class for unseen categories
    unique_values = list(df[col].unique()) + ['unseen_category']
    encoders[col] = LabelEncoder().fit(unique_values)
    
    # Transform training data
    df[col] = encoders[col].transform(df[col])
    
    # Transform test data, replacing unseen categories with 'unseen_category'
    df_test[col] = df_test[col].apply(lambda x: x if x in encoders[col].classes_ else 'unseen_category')
    df_test[col] = encoders[col].transform(df_test[col])


## Imputation

For this iteration of the notebook we will be imputing with KNNImputer, in previous iterations I used just a simple median and mode imputation and haven't seen much performance gain over.

In [ ]:
cat_features = [col for col in df.columns if col not in numeric_features and col != 'SalePrice']

X = df.drop(columns=['SalePrice'], axis=1)
X_test = df_test

imputer = KNNImputer()

X_imputed = imputer.fit_transform(X)
X_test_imputed = imputer.transform(X_test.drop(['Id'], axis=1))

df.iloc[:, df.columns != 'SalePrice'] = X_imputed
df_test.iloc[:, df_test.columns != 'Id'] = X_test_imputed


## Feature Selection
We will employ a Decision Trees model to evaluate feature importance, which assists in identifying the most influential variables. With the feature importances we can select which features should be selected for our model defined by some threshold. A common threshold is usually above $.01$ however this parameter can be tuned for different results.

In [ ]:
# Split data, scale it, fit to model and get important features
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = DecisionTreeRegressor(random_state=42)  

model.fit(X_train, y_train)
importances = model.feature_importances_
feature_names = X.columns

feature_importances = dict(zip(feature_names, importances))

In [ ]:
# Filter out variables that have little influence
importance_threshold = 0.0

important_features = {feature: importance for feature, importance in feature_importances.items() if importance >= importance_threshold}

df = df[list(important_features.keys())]

df

In [ ]:
filtered_cat_features = [cat for cat in cat_features if cat in df.columns]
filtered_numeric_features = [num for num in numeric_features if num in df.columns] 
filtered_features = filtered_cat_features + filtered_numeric_features

In [ ]:
df_test = df_test[filtered_features + ['Id']]

In [ ]:
test_cat_features = [col for col in filtered_features if col in cat_features]
test_numeric_features = [col for col in filtered_features if col not in cat_features]

In [ ]:
filtered_cat_features, filtered_numeric_features

In [ ]:
test_cat_features, test_numeric_features

In [ ]:
(len(filtered_cat_features), len(filtered_numeric_features)), (len(test_cat_features), len(test_numeric_features))

In [ ]:
df['SalePrice'] = y

# Model Building

We are using a PyTorch Dataset which will convert our features into tensors which can be easily indexed. This approach is more modular and provides efficient data loading. If you're not familiar with PyTorch datasets you need to define 3 basic functions as well as inherit the superclass Dataset. The reason we added the additional logic for the case when target is None is for our submission dataset where that column is omitted.

## Creating Tensors
- Convert dataframes into numpy arrays 
- Convert numpy arrays into Tensors
    - Create tensors (train, test) for Numerical Columns (float)
    - Create tensors (train, test) for Categorical Cumns (int64)
    - Create tensor for Target Variable (SalePrice)


In [ ]:
class HouseDataset(Dataset):
    def __init__(self, df, numeric_features, categorical_features, target=None):
        self.df = df
        self.numeric = torch.tensor(np.stack([df[num].values for num in numeric_features], axis=1), dtype=torch.float)
        self.categorical = torch.tensor(np.stack([df[cat].values for cat in categorical_features], axis=1), dtype=torch.int64)
        
        if target is not None:
            self.target = torch.tensor(df[target].values, dtype=torch.float).reshape(-1, 1)
        else:
            self.target = None

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if self.target is not None:
            return self.categorical[idx], self.numeric[idx],  self.target[idx]
        else:
            return self.categorical[idx], self.numeric[idx]

In [ ]:
df, df_testing = train_test_split(df, test_size=0.1)

In [ ]:
dataset = HouseDataset(df, filtered_numeric_features, filtered_cat_features, 'SalePrice')
test_dataset = HouseDataset(df_testing, filtered_numeric_features, filtered_cat_features, 'SalePrice')
submit_dataset = HouseDataset(df_test, test_numeric_features, test_cat_features)

## Embedding Sizes 
Since the train and test columns result in varying amount of unique values we will need to get the embedding sizes for both the train and test. This will change how we compute the final embeddings.

In [ ]:
t_embedding_sizes = [len(df_test[col].unique()) for col in df_test[test_cat_features]]
print(t_embedding_sizes)

In [ ]:
embedding_sizes = [len(df[col].unique()) for col in df[filtered_cat_features]]
print(embedding_sizes)

Here we are taking the max between the two different embeddings to ensure we don't have shape issues when training or predicting.

In [ ]:
embedding_dimensions = [(max(size, t_size)+1, min(50, (max(size, t_size)+1) // 2)) for size, t_size in zip(embedding_sizes, t_embedding_sizes)]
embedding_dimensions

## Model Architecture

### Initialization

In the initialization phase we are defining two types of layers
- **Embedding Layer**
    - We create a layer for each embedding dimension defined earlier
    - Define Dropout Layer for the embeddings 
    - Define Batch Normalization layer for embeddings 

- **Linear Layers**
    - We create a series of layers dependant on the layers number passed
    - For each layers we define a fully-connected Linear Layer accompanied by ReLU activation function
    - Define Dropout layer for current Linear Layer
    - Define Batch Normalization for current Linear Layer
    
### Forward Passes

- For the forward pass we give the model the categorical and continous features for that batch. The model first calculates the embeddings based on the categorical features and embedding layers defined earlier. The continous features are then passed into the Batch Normalization to Standardize them before being concatenated with the embedded categorical features. The concatenated batch is then sent through the series of Fully-Connected Linear Layers.

In [ ]:
class RegressionModel(nn.Module):
    def __init__(self, embedding_dimensions, num_continuous, out_size, layers, dropout_rate=0.4):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(input_dim, output_dim) for input_dim, output_dim in embedding_dimensions])
        self.embeddings_dropout = nn.Dropout(dropout_rate)
        self.bn_continuous = nn.BatchNorm1d(num_continuous)
        
        layers_list = []
        num_embeddings = sum((output for _, output in embedding_dimensions))
        total_in = num_embeddings + num_continuous
        
        for i, layer in enumerate(layers):
            layers_list.append(nn.Linear(total_in, layer))
            layers_list.append(nn.ReLU(inplace=True))
            layers_list.append(nn.BatchNorm1d(layer))
            curr_dropout_rate = dropout_rate * (len(layers) - i) / len(layers)
            layers_list.append(nn.Dropout(curr_dropout_rate))
            total_in = layer
            
        layers_list.append(nn.Linear(layers[-1], out_size))
        self.layers = nn.Sequential(*layers_list)
            
    def forward(self, categorical, continuous):
        embeddings = []
        for i, e in enumerate(self.embeddings):
            embeddings.append(e(categorical[:, i]))
        x = torch.cat(embeddings, 1)
        x = self.embeddings_dropout(x)
        
        cont = self.bn_continuous(continuous)
        x = torch.cat([x, cont], 1)
        x = self.layers(x)
        return x

In [ ]:
model = RegressionModel(embedding_dimensions, len(filtered_numeric_features), 1, [250, 100,50], 0.4)
model = model.to(device)

In [ ]:
model

This is a nice representation of our model, we can see the embedding layers defined by the dimensions we created earlier. Then we can see the series of linear layers we defined.

## Model Training

In the following cell, we make a training loop with several components. The first one to mention is we are using KFold Cross Validation to find the best pair of folds for training and validation. We also are using early stopping here in order to stop the model once loss has stopped decreasing. We are also training over a large number of epochs with a fast learning rate, this is due to our scheduler. We have a ReduceLROnPlateau which will reduce the learning rate once the model stops improving which allows us to use a high learning rate. In addition we are using RMSE as our loss function. 

In [ ]:
EPOCHS = 500
patience = 5
kfold = KFold(n_splits=5, shuffle=True)

best_overall_accuracy = 0.0 
best_overall_loss = float('inf')
best_model_state = None

# Lists to track training and validation losses over all epochs and folds
all_train_losses = []
all_val_losses = []

# Begin k-fold cross-validation.
for fold, (train_idx, val_idx) in enumerate(kfold.split(df.index, df['SalePrice'])):
    print(f"Starting fold {fold+1}")
    df_train, df_val = df.iloc[train_idx], df.iloc[val_idx]
    
    train_dataset = HouseDataset(df_train, filtered_numeric_features, filtered_cat_features, 'SalePrice')
    val_dataset = HouseDataset(df_val, filtered_numeric_features, filtered_cat_features, 'SalePrice')
    
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
    
    model = RegressionModel(embedding_dimensions, len(filtered_numeric_features), 1, [100,50], 0.4)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.1)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)
    loss_function = nn.MSELoss()

    best_val_loss = float('inf')
    epochs_no_improve = 0
    early_stopping_patience = 250

    # Lists to track per-fold training and validation losses
    fold_train_losses = []
    fold_val_losses = []

    # Begin training for a set number of epochs.
    for epoch in range(EPOCHS):
        model.train()
        train_loss_epoch = 0
        for cat, cont, y in train_loader:
            cat, cont, y = cat.to(device), cont.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred = model(cat, cont)
            loss = torch.sqrt(loss_function(y_pred, y))
            loss.backward()
            optimizer.step()
            train_loss_epoch += loss.item()

        train_loss_epoch /= len(train_loader)
        fold_train_losses.append(train_loss_epoch)  # Track training loss for the current fold

        model.eval()
        val_loss_epoch = 0
        with torch.no_grad():
            for val_cat, val_cont, val_y in val_loader:
                val_cat, val_cont, val_y = val_cat.to(device), val_cont.to(device), val_y.to(device)
                val_y_pred = model(val_cat, val_cont)
                val_loss = torch.sqrt(loss_function(val_y_pred, val_y))
                val_loss_epoch += val_loss.item()

        val_loss_epoch /= len(val_loader)
        fold_val_losses.append(val_loss_epoch)  # Track validation loss for the current fold

        if val_loss_epoch < best_val_loss:
            best_val_loss = val_loss_epoch
            epochs_no_improve = 0
            best_model_state = model.state_dict().copy()
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= early_stopping_patience:
                print(f"Early stopping triggered. Stopping training at fold {fold+1}, epoch {epoch+1}")
                break

        scheduler.step(val_loss_epoch)

    # Append per-fold losses to the overall tracking lists
    all_train_losses.append(fold_train_losses)
    all_val_losses.append(fold_val_losses)

    if best_val_loss < best_overall_loss:
        best_overall_loss = best_val_loss
        best_model_state = model.state_dict().copy()

torch.save(best_model_state, 'best_model.bin')
print("Best model saved with overall lowest validation loss.")

In [ ]:
# Save the best model from the best fold
best_model_path = 'best_model.bin'
model.load_state_dict(torch.load(best_model_path))
model.to(device) 

In [ ]:
def pad_with_nan(list_of_lists):
    max_length = max(len(single_list) for single_list in list_of_lists)
    return [single_list + [np.nan]*(max_length - len(single_list)) for single_list in list_of_lists]

padded_train_losses = pad_with_nan(all_train_losses)
padded_val_losses = pad_with_nan(all_val_losses)

padded_train_losses_array = np.array(padded_train_losses)
padded_val_losses_array = np.array(padded_val_losses)

average_train_losses = np.nanmean(padded_train_losses_array, axis=0)
average_val_losses = np.nanmean(padded_val_losses_array, axis=0)

plt.figure(figsize=(10, 6))
plt.plot(average_train_losses, label='Average Training Loss', color='teal')
plt.plot(average_val_losses, label='Average Validation Loss', color='coral')
plt.title('Average Training and Validation Losses Across Folds')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.legend()
plt.show()

This graph shows that we slowly decreased both losses over a lot of epochs. Usually the graph slows down at around 2500 which will be displayed here. If the epochs is set low this graph will usually have a lot of variance which indicates our model has not converged yet.

## Model Evaluation and Submission

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False) 

model.eval()

test_loss = 0
all_y_pred = []
all_y_true = []

with torch.no_grad():
    for batch in test_loader:
        cat, cont, y_true = batch
        cat, cont, y_true = cat.to(device), cont.to(device), y_true.to(device)
        y_pred = model(cat, cont)

        loss = torch.sqrt(loss_function(y_pred, y_true))
        test_loss += loss.item()

        all_y_pred.append(y_pred)
        all_y_true.append(y_true)

all_y_pred = torch.cat(all_y_pred, dim=0)
all_y_true = torch.cat(all_y_true, dim=0)

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss}")

mse = mean_squared_error(all_y_true.cpu().numpy(), all_y_pred.cpu().numpy())
rmse = np.sqrt(mse)
r2 = r2_score(all_y_true.cpu().numpy(), all_y_pred.cpu().numpy())

print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared: {r2}")

In [ ]:
submit_loader = DataLoader(submit_dataset, batch_size=64, shuffle=False)

In [ ]:
model.eval()

test_loss = 0
all_y_pred = []

with torch.no_grad():
    for cat, cont in submit_loader:
        cat, cont = cat.to(device), cont.to(device)
        y_pred = model(cat, cont)
        all_y_pred.append(y_pred)
        
all_y_pred = torch.cat(all_y_pred).cpu().numpy()

df_submit = pd.DataFrame({
    'Id': df_test['Id'],
    'SalePrice': all_y_pred.flatten()  
})

df_submit.to_csv('submission.csv', index=False)